In [ ]:
import math
from collections import defaultdict
import json
import csv
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import tweepy
# We must use the API to determine whether the tweets are protected
from tweepy import TweepError

In [ ]:
consumer_key = "PQSsqSxGft69D53XGknXya8cL"
consumer_secret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
access_token = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
access_token_secret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
'''
This is a test run for just the Iraq earthquake event (it is smaller and easier to work with)
'''
originalUserHistories = defaultdict(list)
userTweetDict = defaultdict(str)
tweetText = defaultdict(str)
origTweets = defaultdict(str)

# Read in the JSON data to get the text of each tweet and build user histories
#TODO: Change the filepath here
with open("/Users/chloelarkin/Desktop/iraq_earthquake_final_data.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user = json_data['user']['id'] # Get the user ID
        tweet_id = json_data['id']
        origTweets[tweet_id] = user
        userTweetDict[tweet_id] = user
        tweet_text = json_data['text']  # Get the text of the tweet
        tweetText[tweet_id] = tweet_text
        originalUserHistories[user].append(tweet_id)
        # try: #TODO: This is where we can filter out private tweets
        #     u = api.get_user(user) # Check if the user's tweets are protected. If they are public, then append to our dataset
        #     if not u.protected:
        #         tweet_id = json_data['id']
        #         userTweetDict[tweet_id] = user
        #         tweet_text = json_data['text'] # Get the text of the tweet
        #         originalTweetText[user].append(tweet_text)
        #         originalTweetIDs[user].append(tweet_id)
        # except TweepError as e:
        #     if e.args[0][0]['code'] == 63: # The error code for suspended users. Skip these people
        #         continue

In [ ]:
# Extract tweet histories from original dataset if a user has multiple tweets in the original dataset
indTweetsHistories = defaultdict(list) # This stores the tweet IDs of other tweets from that user in the original dataset
for user in originalUserHistories:
    if len(originalUserHistories[user]) > 1: # Check that the user has written more than one tweet
        for tweet in originalUserHistories[user].copy():
            copyExtraTweets = originalUserHistories[user].copy()
            copyExtraTweets.pop(copyExtraTweets.index(tweet))
            indTweetsHistories[tweet] = copyExtraTweets

# Next, build up a history for each user from the extra tweet corpus
extraTweets = defaultdict(list)
with open("/Users/chloelarkin/Desktop/iraq_earthquake_extras.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user = json_data['user']['id'] # Get the user ID
        if user in originalUserHistories: # Only collect tweets from users in the MMD dataset
            tweet_id = json_data['id']
            tweet_text = json_data['text'] # Get the text of the tweet
            tweetText[tweet_id] = tweet_text # Continue building dict from tweet ID to string text
            extraTweets[user].append(tweet_id) # This appends the tweet ID to the extraTweets[user] list

# Get rid of any duplicates between the original dataset and the extra dataset
for tweet in extraTweets[user]:
    if tweet in origTweets.keys():
        extraTweets[user].remove(tweet)

In [ ]:
# Create tweetHistory dict storing the user's history for each labeled tweet (exclusive of the current labeled tweet)
historyDocs = defaultdict(str)
for tweet in origTweets: # this dict refers to all tweets in the dataset
    user = userTweetDict[tweet] # look up which user the tweet belonged to so that you can find the extra tweets
    if len(indTweetsHistories) > 0: # Case where there is at least 1 other tweet from the same author in CrisisMMD
        if len(extraTweets[user]) > 0:
            for newTweet in extraTweets[user]: # Find the extra tweets for that user
                indTweetsHistories[tweet].append(newTweet) # Append them to this tweet history
        listTweets = []
        for entry in indTweetsHistories[tweet]:
            listTweets.append(tweetText[entry])
        if len(listTweets) > 0:
            historyDocs[tweet] = ' '.join(listTweets)
    else: # Case where there are no tweets from the same user in CrisisMMD, but there are among the tweets we extracted
        if len(extraTweets[user]) > 0:
            listTweets = []
            for newTweet in extraTweets[user]:
                listTweets.append(newTweet)
            if len(listTweets) > 0:
                historyDocs[tweet] = ' '.join(listTweets)
tweets = list(key for key in historyDocs.keys())
corpus = list(value for value in historyDocs.values())

In [ ]:
# Get TF-IDF vectors for each tweet in the original corpus
tfidfVectors = defaultdict(list)  # format:  tfidfVectors[user] = tf-idf arr<>
vectorizer = TfidfVectorizer()

# Construct TF-IDF vectors for each user history from ALL EXTRA TWEETS OF USERS IN THE ORIGINAL AND NEW DATASET
vectors = vectorizer.fit_transform(corpus)
for i in range(len(tweets)):
    tfidfVectors[i] = vectors[i] # Map vectors to their users
print("# of TFIDF vectors:", len(tfidfVectors)) # Test to make sure there are the correct number of vectors

print("There were", len(tweetText), "total tweets,", len(origTweets),
      "tweets in the CrisisMMD dataset, and ", len(historyDocs), "tweets with histories.")

In [ ]:
# Establish dicts of annotated scores for each tweet ID
text_informative = defaultdict(str) #4000 scores -- this is the binary informative / not informative score
text_humanitarian = defaultdict(str) # 4000 scores -- finer-grain classification

# Read in the classifications of each tweet
with open("/Users/chloelarkin/Desktop/iraq_iran_earthquake_final_data.tsv") as fin:
    reader = csv.DictReader(fin, dialect='excel-tab')
    for line in reader:
        text_informative[line['tweet_id']]=line['text_info']
        text_humanitarian[line['tweet_id']]=line['text_human']

# Retrieve scores for each tweet in the dataset
informative_scores = defaultdict(str)
humanitarian_scores = defaultdict(str)
for id in origTweets.keys():
    informative_scores[id] = text_informative["\""+ str(id) + "\""]
for id in origTweets.keys():
    humanitarian_scores[id] = text_humanitarian["\""+ str(id) + "\""]

# Sanity check to make sure there is one score for each tweet in the CrisisMMD set
print("There are", len(origTweets), "tweets and", len(informative_scores),
      "informative scores and", len(humanitarian_scores), "humanitarian scores.")